In [1]:
import os
import numpy as np

from utilities import PendulumPlant_, DirectCollocationCalculator, LQRController,\
                     TVLQRController, prepare_trajectory, SOSequalityConstrained
from SosBilinearAlternation import TVmultSearch, TVrhoSearch

# Parameters initialization

In [2]:
log_dir = "log_data/direct_collocation"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# pendulum parameters
mass = 0.57288
length = 0.5
damping = 0.15
gravity = 9.81
coulomb_fric = 0.0
torque_limit = 1.5

# swingup parameters
x0 = [0.0, 0.0]
goal = [np.pi, 0.0]

# direct collocation parameters, N is chosen also to be the number of knot points
N = 30 
max_dt = 0.1

# Compute the nominal trajectory via direct collocation

In [3]:
dircal = DirectCollocationCalculator()
dircal.init_pendulum(mass=mass,
                     length=length,
                     damping=damping,
                     gravity=gravity,
                     torque_limit=torque_limit)
x_trajectory, dircol, result = dircal.compute_trajectory(N=N,
                                                         max_dt=max_dt,
                                                         start_state=x0,
                                                         goal_state=goal)
T, X, XD, U = dircal.extract_trajectory(x_trajectory, dircol, result, N=N)

# save results
csv_data = np.vstack((T, X, XD, U)).T
csv_path = os.path.join(log_dir, "trajectory.csv")
np.savetxt(csv_path, csv_data, delimiter=',',
           header="time,pos,vel,torque", comments="")

# load results
csv_path = "log_data/direct_collocation/trajectory.csv"
data_dict = prepare_trajectory(csv_path)
trajectory = np.loadtxt(csv_path, skiprows=1, delimiter=",")
time = trajectory.T[0].T
dt = time[1]-time[0]
x0_traj = [trajectory.T[1].T, trajectory.T[2].T]

# Funnel Computation

In [4]:
def TVSOSrhoVerification(pendulum, controller, x0_traj, time, N, rhof):

    # Initial rho(t) definition # TODO: initial rho computation (exponential init)
    rho_t = np.ones(N)
    rho_t[-1] = rhof
    
    # Bilinear SOS alternation for improving the first guess
    convergence = False
    eps = 0.1
    while(not convergence):
        for knot in np.flip([i for i in range(1,N)]):
            
            # min value of rho[knot]
            rho_max = 0.0

            # Search for a multiplier, fixing rho
            fail = True
            while (fail):
                (fail, Q) = TVmultSearch(pendulum, controller, x0_traj, knot, time, rho_t)
                if fail:
                    rho_t[knot-1] = 0.75*rho_t[knot-1]
                    print(f"Mult step Error in knot {knot-1}")

            print("---------------")
            print(Q)
            print("---------------")

            # Search for rho, fixing the multiplier        
            fail = True
            while (fail):
                (fail, rho_opt) = TVrhoSearch(pendulum, controller, x0_traj, knot, time, Q, rho_t)
                if fail:
                    #rho_t[knot-1] = 0.75*rho_t[knot-1]
                    #(mul_fail, Q) = TVmultSearch(pendulum, controller, x0_traj, knot, time, rho_t)
                    print(f"Rho step Error in knot {knot-1}")

            rho_max = max(rho_opt, rho_max)
            rho_t[knot-1] = rho_max
            
        print("---------------")
        print(rho_t)
        print("---------------")
        
        # Check for convergence
        if((np.sum(rho_t) - np.sum(rho_t_prev))/np.sum(rho_t_prev) < eps): 
            convergence = True
        rho_t_prev = rho_t

    return (rho_t, controller.tvlqr.S)


In [ ]:
pendulum = PendulumPlant_(mass=mass,
                         length=length,
                         damping=damping,
                         gravity=gravity,
                         coulomb_fric=coulomb_fric,
                         inertia=None,
                         torque_limit=torque_limit)

controller = TVLQRController(data_dict=data_dict, mass=mass, length=length,
                             damping=damping, gravity=gravity,
                             torque_limit=torque_limit)

# Taking the finals values of S and rho from the invariant case, SOS method has been chosen
(rhof, Sf) = SOSequalityConstrained(pendulum,LQRController(mass=mass,
                                                            length=length,
                                                            damping=damping,
                                                            gravity=gravity,
                                                            torque_limit=torque_limit))
controller.set_Qf(Sf)
controller.set_goal(goal)
S_t = controller.tvlqr.S

# Application of the algorithm for time-variand RoA estimation
(rho, S) = TVSOSrhoVerification(pendulum, controller, x0_traj, time, N, rhof)